In [1]:
#!/usr/bin/env python

!pip3 install fuzzywuzzy

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
data = pd.read_csv("ratings.csv")

C:\Users\vishw\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [15]:
books = pd.read_csv('books_enriched.csv', index_col = 'book_id')

In [6]:
#pivot ratings into book features
df_book_features = pd.pivot_table(data=data,
    index='book_id',
    columns='user_id',
    values='rating',fill_value=0)
title_list=df_book_features.index.tolist()
mat_book_features = csr_matrix(df_book_features.values)

In [7]:
mat_book_features.todense()

matrix([[0, 0, 0, ..., 4, 4, 4],
        [0, 5, 0, ..., 5, 5, 5],
        [0, 0, 0, ..., 0, 0, 4],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [18]:
books.loc[df_book_features.index[10], 'title']

'The Kite Runner'

In [38]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(mat_book_features)
test_title=input("What book have you recently read: ")
query_index=title_list.index(process.extract(test_title,title_list)[0][0])
distances, indices = model_knn.kneighbors(df_book_features.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)
for i in range(0,len(distances.flatten())):
    if i ==0:
        print('Recommendations for {0}:\n'.format(books.loc[df_book_features.index[query_index], 'title']))
    else:
        print('{0}:{1}, with distance of {2}.'.format(i,books.loc[df_book_features.index[indices.flatten()[i]], 'title'],distances.flatten()[i]))

print(' \n')        
model_knn2 = NearestNeighbors(metric='euclidean', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn2.fit(mat_book_features)
distances, indices = model_knn2.kneighbors(df_book_features.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)
for i in range(0,len(distances.flatten())):
    if i ==0:
        print('Recommendations for {0}:\n'.format(books.loc[df_book_features.index[query_index], 'title']))
    else:
        print('{0}:{1}, with distance of {2}.'.format(i,books.loc[df_book_features.index[indices.flatten()[i]], 'title'],distances.flatten()[i]))


What book have you recently read: 92
Recommendations for Freakonomics: A Rogue Economist Explores the Hidden Side of Everything (Freakonomics, #1):

1:The Tipping Point: How Little Things Can Make a Big Difference, with distance of 0.590511108092088.
2:Blink: The Power of Thinking Without Thinking, with distance of 0.6401722564691626.
3:SuperFreakonomics: Global Cooling, Patriotic Prostitutes And Why Suicide Bombers Should Buy Life Insurance, with distance of 0.6480742106136235.
4:Outliers: The Story of Success, with distance of 0.6575011431305207.
5:The World Is Flat: A Brief History of the Twenty-first Century, with distance of 0.7481699997987048.
 

Recommendations for Freakonomics: A Rogue Economist Explores the Hidden Side of Everything (Freakonomics, #1):

1:SuperFreakonomics: Global Cooling, Patriotic Prostitutes And Why Suicide Bombers Should Buy Life Insurance, with distance of 256.0976376306505.
2:The World Is Flat: A Brief History of the Twenty-first Century, with distance o